In [1]:
import torch
import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [3]:
# def f(x):
#     return x**2 + np.sin(x) + np.exp(x)

In [4]:
# def integral(f, a, b, N):
#     deltax = (b-a)/N
#     f_v = np.vectorize(f)
#     f_evals = f_v(np.linspace(a, b, N+1, endpoint=True))
# #     f_evals = np.array([f_v(i) for i in np.linspace(a, b, N+1, endpoint=True)])
#     value = deltax*(f_evals[0] + 2*f_evals[1:-1].sum() + f_evals[-1])/2
#     return value
# # print((b**3 - a**3)/3)matplotlib

In [5]:
# import matplotlib.pyplot as plt

In [6]:
# n_vals = range(5, 100, 5)
# evals = [integral(f, 0, 10.5, n) for n in n_vals]
# plt.plot(n_vals, evals)

In [7]:
# print(evals)

In [8]:
cd '/Users/arighosh/Desktop/git/hierarchical_point_process/src/'

/Users/arighosh/Desktop/git/hierarchical_point_process/src


In [10]:
from hrmtpp_exact import hrmtpp_exact
from rmtpp import rmtpp

In [11]:
model = rmtpp(marker_dim=217).to(device)

In [46]:
from utils.data_loader import load_data

mimic_x, mimic_t = load_data('../data/mimic_train.pkl')
(mimic_t[8])

array([[1.63606126e+02, 2.66482242e-02, 1.63606126e+02],
       [1.63673353e+02, 3.03939463e-03, 6.72275458e-02],
       [1.66307445e+02, 5.74403630e-03, 2.63409195e+00],
       [1.67025299e+02, 7.27111694e-03, 7.17853069e-01],
       [1.67101833e+02, 1.29078257e-02, 7.65343372e-02],
       [1.67234063e+02, 6.24635587e-03, 1.32230588e-01]])

In [50]:
#First time value is the offset. High value. Removing that. Time interval column  is the last one.
t = torch.from_numpy(np.stack([mimic_t[0][1:5,:], mimic_t[3][1:5,:]], axis=1)).float().to(device)[:,:,[0,2]]
x = torch.from_numpy(np.stack([mimic_x[0][1:5,:], mimic_x[3][1:5,:]], axis=1)).float().to(device)
x.shape, t.shape, t

(torch.Size([4, 2, 217]),
 torch.Size([4, 2, 2]),
 tensor([[[2.0149e+02, 8.5619e-01],
          [1.4684e+02, 4.4446e-01]],
 
         [[2.0197e+02, 4.7978e-01],
          [1.4685e+02, 9.5616e-03]],
 
         [[2.0284e+02, 8.6874e-01],
          [1.4955e+02, 2.6970e+00]],
 
         [[2.0306e+02, 2.1820e-01],
          [1.4984e+02, 2.8553e-01]]]))

In [107]:
def compute_point_log_likelihood(h, d_js):
        """
            Input:
                h : Tensor of shape TxBSxCx self.shared_output_layers[-1]
                tj : Tensor of shape TxBSx2 [j,:,0] represents actual time at timestep j ,\
                    [i,:,1] represents time gap d_i = t_i- t_{i-1}
                d_js: Tensor of shape TxBSx1x(N+1) [j,:,:,k] represents the k*delta_x in trapezoidal rule
                [j,:,:,0] is 0
                https://en.wikipedia.org/wiki/Trapezoidal_rule#Uniform_grid
            Output:
                log_f_t : tensor of shape TxBSxC

        """
        #hz_embedded shape TxBSx64, d_js shape is TxBSx1xN
        
        
        #d_js = tj[:, :, 0:1, None]  # Shape TxBSx1xN Time differences

        past_influence = model.h_influence(h)[:,:,:,None]  # TxBSx1,x1


        # TxBSx1x1
        current_influence = model.time_influence * d_js#TxBSx1xN
        base_intensity = model.base_intensity[:,:,:, None]  # 1x1x1x1
        #print(past_influence.shape,current_influence.shape, base_intensity.shape)

        term1 = past_influence + current_influence + base_intensity
        term2 = (past_influence + base_intensity).exp()
        term3 = term1.exp()

        log_f_t = term1 + \
            (1./(model.time_influence+1e-6 )) * (term2-term3)
        return log_f_t  # TxBSxCx(N+1)

    
with torch.no_grad():
    #Only need interval
    a = t[:, :, 1][:,:, None, None] #TxBSx1x1
    #Increase N, for correct integration
    N = 10000
    #interval can be anything from 0 to some constant * maximum interval length observed
    dmax =  5* a.max() #- a.min()
    delta_x = dmax/N
    d_js = (torch.arange(0, N+1).float().to(device)*delta_x)[None,None,None,:]#1x1x1xN
    #Add a dummy row for the actual time
    #Interval starts from 0 to max
    repeat_val = (4,2,-1,-1)
    d_js = d_js.expand(*repeat_val)#TxBSx1xN
    print(d_js.shape)
    times = d_js
    hs, hz_embedded = model.run_forward_rnn(x, t)
    hz_embedded = hz_embedded[:-1,:,:]
    print ("intervals are: ", a[:,:,0,0])
    #print(times)
    #hz_embedded = model.preprocess_hidden_latent_state(hs)
    #hz_embedded shape TxBSx64, d_js shape is TxBSx1xN
    time_log_likelihood = compute_point_log_likelihood(hz_embedded, d_js) #TxBSx1xN
    #print(time_log_likelihood)#TxBSx1xN
    time_likelihood = time_log_likelihood.exp()[:,:,0,:]#TxBSxN (Maybe should be using some stable version of that)
    
    #Check whether prob sums to 1 or not. If sum< 1, increase integration max limit. If sum>1. Increase N.
    sum_probs = time_likelihood.sum(dim =[2])*delta_x
    print("Sum of probs are: ", sum_probs)
    
    #Compute Expectation
    #t*f(t)
    g = time_likelihood * d_js[:,:,0,:]*delta_x# TxBSxN
    expectation = g.sum(dim =-1)
    print("Expected output is ", expectation)


torch.Size([4, 2, 1, 10001])
intervals are:  tensor([[0.8562, 0.4445],
        [0.4798, 0.0096],
        [0.8687, 2.6970],
        [0.2182, 0.2855]])
Sum of probs are:  tensor([[1.0008, 1.0008],
        [1.0007, 1.0007],
        [1.0008, 1.0008],
        [1.0008, 1.0008]])
Expected output is  tensor([[1.0051, 1.0051],
        [1.0398, 1.0475],
        [1.0082, 1.0171],
        [1.0065, 1.0119]])


IndexError: too many indices for tensor of dimension 3

In [48]:
times.shape, g.shape, l.sum(dim=-1), delta_x

NameError: name 'times' is not defined

In [40]:
integral.shape

torch.Size([3, 2, 5, 1])

In [41]:
time_log_likelihood

tensor([[[[ 7.4486e-03, -1.0391e+02, -3.1650e+02,  ..., -2.0832e+32,
           -4.2468e+32, -8.6578e+32],
          [-3.6657e-02, -9.9441e+01, -3.0283e+02,  ..., -1.9933e+32,
           -4.0636e+32, -8.2842e+32],
          [-2.2586e-01, -8.2372e+01, -2.5058e+02,  ..., -1.6497e+32,
           -3.3631e+32, -6.8562e+32],
          [ 5.7212e-02, -1.0920e+02, -3.3268e+02,  ..., -2.1895e+32,
           -4.4635e+32, -9.0996e+32],
          [-5.7042e-02, -9.7441e+01, -2.9671e+02,  ..., -1.9531e+32,
           -3.9816e+32, -8.1171e+32]],

         [[ 7.4486e-03, -1.0391e+02, -3.1650e+02,  ..., -2.0830e+32,
           -4.2465e+32, -8.6571e+32],
          [-3.6657e-02, -9.9441e+01, -3.0283e+02,  ..., -1.9931e+32,
           -4.0633e+32, -8.2836e+32],
          [-2.2586e-01, -8.2372e+01, -2.5058e+02,  ..., -1.6496e+32,
           -3.3629e+32, -6.8557e+32],
          [ 5.7212e-02, -1.0920e+02, -3.3268e+02,  ..., -2.1893e+32,
           -4.4632e+32, -9.0989e+32],
          [-5.7042e-02, -9.7441e+01

In [42]:
delta_x

tensor([[[[71.2287]],

         [[71.2286]]],


        [[[71.2287]],

         [[71.2287]]],


        [[[71.2287]],

         [[71.2287]]]], device='cuda:0')

In [43]:
dmax

tensor(71228.6328, device='cuda:0')